In [1]:
import os
from PIL import Image
import pandas as pd 
import glob
import random as r
import tqdm
import json

In [2]:
loc = "../../data/"

# jsons from fish_crawlers_nonattached
fish_crawlers_nonattached_labels = glob.glob(loc + "rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled*/export*/*.json")
fish_crawlers_nonattached_labels

['../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_7_251_300/export/export-2020-12-07T06_48_13.484Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_3_port_151_200/export/3_port_151_200.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_11_151_200/export/export-2020-12-07T06_18_45.164Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_11_251_300/export/export-2020-12-07T06_47_45.271Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_3_port_251_300/export/3_port_251_300.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_17_101_150/export/export-2020-12-07T07_18_23.619Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_11_101_150/export/export-2020-12-07T06_11_33.304Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_7_201_250/export/export-2020-12-07T06_30_50.847Z.json',
 '../../data/rockfish_full/S

In [3]:
# jsons for the directory one level up
# labeled_14* is missing but I couldn't find it in the bucket 
she_11_labels = glob.glob(loc + "rockfish_full/SHE-11/Labeled_*/export*/*.json")
she_11_labels

['../../data/rockfish_full/SHE-11/Labeled_8_s_1_100/export/8_stbd_1_100_Joy.json',
 '../../data/rockfish_full/SHE-11/Labeled_21_s_1_100/export_1_100/export-2020-12-05T09_49_04.355Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_7_s_101_150/export/export-2020-12-06T06_31_45.283Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_3_p_1_100/export/3_port_1_100.json',
 '../../data/rockfish_full/SHE-11/Labeled_17_p_1_100/export/export-2020-12-06T07_05_18.687Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_11_S_1_100/export/export-2020-12-06T06_37_13.628Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_7_s_1_100/export/export-2020-12-06T06_31_32.231Z.json']

In [4]:
# remove labeled_8* and _3* from SHE-11 but not from fishcrawlers 
for l in she_11_labels:
    if "Labeled_8" in l:
        she_11_labels.remove(l)
    elif "Labeled_3" in l:
        she_11_labels.remove(l)
    
    
she_11_labels

['../../data/rockfish_full/SHE-11/Labeled_21_s_1_100/export_1_100/export-2020-12-05T09_49_04.355Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_7_s_101_150/export/export-2020-12-06T06_31_45.283Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_17_p_1_100/export/export-2020-12-06T07_05_18.687Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_11_S_1_100/export/export-2020-12-06T06_37_13.628Z.json',
 '../../data/rockfish_full/SHE-11/Labeled_7_s_1_100/export/export-2020-12-06T06_31_32.231Z.json']

In [5]:
labels = fish_crawlers_nonattached_labels + she_11_labels
labels

['../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_7_251_300/export/export-2020-12-07T06_48_13.484Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_3_port_151_200/export/3_port_151_200.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_11_151_200/export/export-2020-12-07T06_18_45.164Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_11_251_300/export/export-2020-12-07T06_47_45.271Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_3_port_251_300/export/3_port_251_300.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_17_101_150/export/export-2020-12-07T07_18_23.619Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_11_101_150/export/export-2020-12-07T06_11_33.304Z.json',
 '../../data/rockfish_full/SHE-11/Fish_Crawlers_NonAttached/Labeled_7_201_250/export/export-2020-12-07T06_30_50.847Z.json',
 '../../data/rockfish_full/S

4 missing images

In [6]:
total_ims = 0
total_bboxes = 0
missing = 0
missing_ims = []
for l in labels:
    label_list = json.load(open(l, "r"))
    total_ims += len(label_list)
    for im in label_list:
        if 'objects' in im['Label']:
            bboxes = im['Label']['objects']
            total_bboxes += len(bboxes)
        else:
            missing += 1
            missing_ims.append(im)
print(total_ims, "Total images")
print(total_bboxes, "Total bboxes")
print(missing, "missing: ", [im['External ID'] for im in missing_ims])

1433 Total images
15012 Total bboxes
4 missing:  ['20111015.204550.00189_uncropped_rect_color.jpg', '20111012.211108.00037_uncropped_rect_color.jpg', '20111008.171203.00443_uncropped_rect_color.png', '20111015.165052.00593_uncropped_rect_color.jpg']


In [8]:
def get_image_path(labels_path):
    """
    Takes path to labels and truncates it to path to images.
    I need this to append image paths to each image file.
    """
    split_lp = os.path.dirname(labels_path).split("/")
    split_lp = split_lp[2:-1]
    rejoined = "/".join(split_lp)
    
    return rejoined

In [9]:
box_list = []
missing = 0
for l in labels:
    images_path = get_image_path(l)
    label_list = json.load(open(l, "r"))
    for _, label in enumerate(label_list):
        filename = label['External ID']
        if 'objects' in label['Label']:
            bboxes = label['Label']['objects']
            for box in bboxes:
                x_min = box['bbox']['left']
                y_min = box['bbox']['top']
                x_max = x_min + box['bbox']['width']
                y_max = y_min + box['bbox']['height']
                box_list.append({
                    "x_min": x_min,
                    "x_max": x_max,
                    "y_min": y_min,
                    "y_max": y_max,
                    "filename": os.path.join(images_path, filename),
                    "title": box['title'],
                    "value": box['value']
                })
        else: missing += 1
             

print(missing, "missing")
jimmy_df = pd.DataFrame(box_list)

4 missing


In [19]:
# remove /data from beginning of filename paths
def remove_data_from_path(fn):
    split_dirs = fn.split("/")
    split_dirs = split_dirs[1:]
    rejoin = "/".join(split_dirs)
    
    return rejoin

In [20]:
jimmy_df['filename'] = jimmy_df.filename.apply(remove_data_from_path)

jimmy_df

,x_min,x_max,y_min,y_max,filename,title,value
0,244,311,1860,1929,rockfish_full/SHE-11/Fish_Crawlers_NonAttached...,Animal,animal
1,734,876,705,958,rockfish_full/SHE-11/Fish_Crawlers_NonAttached...,Animal,animal
2,1030,1221,1119,1233,rockfish_full/SHE-11/Fish_Crawlers_NonAttached...,No Animal,no_animal
3,1704,1898,823,1020,rockfish_full/SHE-11/Fish_Crawlers_NonAttached...,Animal,animal
4,1224,1308,966,1106,rockfish_full/SHE-11/Fish_Crawlers_NonAttached...,No Animal,no_animal
...,...,...,...,...,...,...,...
15007,1713,1874,1536,1631,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal
15008,1643,1826,1297,1433,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal
15009,1304,1427,665,765,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal
15010,737,943,221,346,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal


In [21]:
jimmy_df.to_csv("jimmy_labels_fixed.csv", index=False)